In [ ]:
Molecular fingerprinting_COVID

In [49]:
# Import required modules
import numpy as np
import pandas as pd
import os
import rdkit as rd
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import RDConfig
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.rdMolDescriptors import GetAtomPairFingerprint
from rdkit.Chem.AtomPairs import Torsions
os.chdir("C:/Users/Welcome/Desktop/MACCS/Main_data")
from collections import Counter

In [50]:
import sklearn as sk
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [51]:
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
leads_data_covid = pd.read_csv("combined_data.csv")


In [53]:
#conver smiles to mol 
leads_data_covid["mol"] = [Chem.MolFromSmiles(x) for x in leads_data_covid["smiles"]]


In [54]:
#morgan
leads_data_covid["morgan_fingerprint"] = [Chem.GetMorganFingerprintAsBitVect(m, 2, nBits = 2048) for m in leads_data_covid['mol']]
morgan_fingerprint = [Chem.GetMorganFingerprintAsBitVect(m, 2, nBits = 2048) for m in leads_data_covid['mol']]
morgan_fingerprint_np = []
for fp in morgan_fingerprint:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  morgan_fingerprint_np.append(arr)

In [55]:
#Daylight fingerprint
leads_data_covid["rdkit_fingerprint"] = [Chem.RDKFingerprint(m) for m in leads_data_covid["mol"]]
rdkit_fingerprint = [Chem.RDKFingerprint(m) for m in leads_data_covid["mol"]]
rdkit_fingerprint_np = []
for fp in rdkit_fingerprint:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  rdkit_fingerprint_np.append(arr)

In [56]:
#Atom-pair fingerprints
leads_data_covid["AtomPair_Fingerprint"] = [Chem.GetHashedAtomPairFingerprintAsBitVect(m) for m in leads_data_covid["mol"]]
AtomPair_Fingerprint = [Chem.GetHashedAtomPairFingerprintAsBitVect(m) for m in leads_data_covid["mol"]]
AtomPair_Fingerprint_np = []
for fp in AtomPair_Fingerprint:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  AtomPair_Fingerprint_np.append(arr)

In [57]:
#torsion fingerprints
leads_data_covid["TorsionFingerprint"] = TorsionFingerprint = [Chem.GetHashedTopologicalTorsionFingerprintAsBitVect(m) for m in leads_data_covid["mol"]]
TorsionFingerprint = [Chem.GetHashedTopologicalTorsionFingerprintAsBitVect(m) for m in leads_data_covid["mol"]]
TorsionFingerprint_np = []
for fp in TorsionFingerprint:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  TorsionFingerprint_np.append(arr)

In [58]:
x_morgan = morgan_fingerprint_np
x_rdkit = rdkit_fingerprint_np
x_Atompair = AtomPair_Fingerprint_np
x_torsion = TorsionFingerprint_np
y = leads_data_covid.active.values

In [60]:
x_morg_adasyn, y_morg_adasyn = ADASYN().fit_resample(x_morgan, y)
morgan_count = sorted(Counter(y_morg_rsmp).items())

x_rd_adasyn, y_rd_adasyn = ADASYN().fit_resample(x_rdkit, y)
rdkit_count = sorted(Counter(y_rd_rsmp).items())

x_AP_adasyn, y_AP_adasyn = ADASYN().fit_resample(x_Atompair, y)
Atompair_count = sorted(Counter(y_rd_rsmp).items())

x_torsion_adasyn, y_torsion_adasyn = ADASYN().fit_resample(x_torsion, y)
torsion_count = sorted(Counter(y_rd_rsmp).items())

print("Morgan data:", morgan_count, "Daylight-like data:", rdkit_count, "Atom-pair data:", Atompair_count, "Torsion data:", torsion_count)


Morgan data: [(0, 1070), (1, 1003)] Daylight-like data: [(0, 1013), (1, 1003)] Atom-pair data: [(0, 1013), (1, 1003)] Torsion data: [(0, 1013), (1, 1003)]


In [61]:
#Logistic Regression
#spliting training & test set
x_morg_train, x_morg_test, y_morg_train, y_morg_test = train_test_split(x_morg_adasyn, y_morg_adasyn, test_size=0.15, random_state=1)
x_morg_train, x_morg_val, y_morg_train, y_morg_val = train_test_split(x_morg_train, y_morg_train, test_size=0.15, random_state=1)
x_rd_train, x_rd_test, y_rd_train, y_rd_test = train_test_split(x_rd_adasyn, y_rd_adasyn, test_size=0.15, random_state=1)
x_rd_train, x_rd_val, y_rd_train, y_rd_val = train_test_split(x_rd_train, y_rd_train, test_size=0.15, random_state=1)
x_AP_train, x_AP_test, y_AP_train, y_AP_test = train_test_split(x_AP_adasyn, y_AP_adasyn, test_size=0.15, random_state=1)
x_AP_train, x_AP_val, y_AP_train, y_AP_val = train_test_split(x_AP_train, y_AP_train, test_size=0.15, random_state=1)
x_torsion_train, x_torsion_test, y_torsion_train, y_torsion_test = train_test_split(x_torsion_adasyn, y_torsion_adasyn, test_size=0.15, random_state=1)
x_torsion_train, x_torsion_val, y_torsion_train, y_torsion_val = train_test_split(x_torsion_train, y_torsion_train, test_size=0.15, random_state=1)

In [62]:
lr = LogisticRegression(solver = 'lbfgs', max_iter = 1000)
accuracy_morg = cross_val_score(lr, x_morg_train, y_morg_train, scoring='accuracy', cv = 10)
print("Accuracy of Model with Cross Validation is:",accuracy_morg.mean() * 100, "(+/- %0.2f)" % accuracy_morg.std())
lr_morg_ = LogisticRegressionCV(cv=10, solver = 'lbfgs', max_iter = 1000).fit(x_morg_train, y_morg_train)

Accuracy of Model with Cross Validation is: 90.18752952290978 (+/- 0.02)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [63]:
def lrCV_model(x_train, y_train):
    lr = LogisticRegression(solver = 'lbfgs', max_iter = 650)
    model_accuracy = cross_val_score(lr, x_train, y_train, scoring='accuracy', cv = 5)
    accuracy_printout = "Accuracy of Model with Cross Validation is:",model_accuracy.mean() * 100, "(+/- %0.2f)" % model_accuracy.std()
    lr_fit = LogisticRegressionCV(cv=5, solver = 'lbfgs', max_iter = 650, multi_class = "ovr", n_jobs = -1).fit(x_train, y_train)
    
    return lr_fit

In [64]:
lr_morg = lrCV_model(x_morg_train, y_morg_train)
lr_rd = lrCV_model(x_rd_train, y_rd_train)
lr_AP = lrCV_model(x_AP_train, y_AP_train)
lr_torsion = lrCV_model(x_torsion_train, y_torsion_train)

In [65]:
predictions_morg_test = lr_morg.predict(x_morg_test)
cm_morg_test = metrics.confusion_matrix(y_morg_test, predictions_morg_test)
print(cm_morg_test)

[[154   6]
 [ 16 127]]


In [66]:
predictions_morg_val = lr_morg.predict(x_morg_val)
cm_morg_val = metrics.confusion_matrix(y_morg_val, predictions_morg_val)
print(cm_morg_val)

[[106   5]
 [ 14 133]]


In [67]:
score_morg_train = lr_morg.score(x_morg_train, y_morg_train)
print('logistic regression classifier_train set:', score_morg_train)

logistic regression classifier_train set: 0.9910775566231984


In [68]:
score_morg_test = lr_morg.score(x_morg_test, y_morg_test)
print('logistic regression classifier_test set:', score_morg_test)

logistic regression classifier_test set: 0.9273927392739274


In [69]:
score_morg_val = lr_morg.score(x_morg_val, y_morg_val)
print('logistic regression classifier_validation:', score_morg_val)

logistic regression classifier_validation: 0.9263565891472868


In [70]:
print(classification_report(y_morg_val, predictions_morg_val))

              precision    recall  f1-score   support

           0       0.88      0.95      0.92       111
           1       0.96      0.90      0.93       147

    accuracy                           0.93       258
   macro avg       0.92      0.93      0.93       258
weighted avg       0.93      0.93      0.93       258



In [71]:
#confusion matrix
predictions_rd_test = lr_rd.predict(x_rd_test)
cm_rd_test = metrics.confusion_matrix(y_rd_test, predictions_rd_test)
predictions_rd_val = lr_rd.predict(x_rd_val)
cm_rd_val = metrics.confusion_matrix(y_rd_val, predictions_rd_val)
print(cm_rd_val)
score_rd_train = lr_rd.score(x_rd_train, y_rd_train)
print('logistic regression classifier_train set:', score_rd_train)
score_rd_test = lr_rd.score(x_rd_test, y_rd_test)
print('logistic regression classifier_test set:', score_rd_test)
score_rd_val = lr_rd.score(x_rd_val, y_rd_val)
print('logistic regression classifier_validation set:', score_rd_val)
print(classification_report(y_rd_val, predictions_rd_val))

[[112   2]
 [ 21 117]]
logistic regression classifier_train set: 0.9929971988795518
logistic regression classifier_test set: 0.8956228956228957
logistic regression classifier_validation set: 0.9087301587301587
              precision    recall  f1-score   support

           0       0.84      0.98      0.91       114
           1       0.98      0.85      0.91       138

    accuracy                           0.91       252
   macro avg       0.91      0.92      0.91       252
weighted avg       0.92      0.91      0.91       252



In [72]:
predictions_AP_test = lr_AP.predict(x_AP_test)
cm_AP_test = metrics.confusion_matrix(y_AP_test, predictions_AP_test)
print(cm_AP_test)
predictions_AP_val = lr_AP.predict(x_AP_val)
cm_AP_val = metrics.confusion_matrix(y_AP_val, predictions_AP_val)
print(cm_AP_val)
score_AP_train = lr_AP.score(x_AP_train, y_AP_train)
print('Accuracy of logistic regression classifier on train set:', score_AP_train)
score_AP_test = lr_AP.score(x_AP_test, y_AP_test)
print('Accuracy of logistic regression classifier on test set:', score_AP_test)
score_AP_val = lr_AP.score(x_AP_val, y_AP_val)
print('Accuracy of logistic regression classifier on validation set:', score_AP_val)
print(classification_report(y_AP_val, predictions_AP_val))

[[145   7]
 [ 19 129]]
[[127   5]
 [ 12 111]]
Accuracy of logistic regression classifier on train set: 0.9923822714681441
Accuracy of logistic regression classifier on test set: 0.9133333333333333
Accuracy of logistic regression classifier on validation set: 0.9333333333333333
              precision    recall  f1-score   support

           0       0.91      0.96      0.94       132
           1       0.96      0.90      0.93       123

    accuracy                           0.93       255
   macro avg       0.94      0.93      0.93       255
weighted avg       0.93      0.93      0.93       255



In [73]:
predictions_torsion_test = lr_torsion.predict(x_torsion_test)
cm_torsion_test = metrics.confusion_matrix(y_torsion_test, predictions_torsion_test)
print(cm_torsion_test)
predictions_torsion_val = lr_torsion.predict(x_torsion_val)
cm_torsion_val = metrics.confusion_matrix(y_torsion_val, predictions_torsion_val)
print(cm_torsion_val)
score_torsion_train = lr_torsion.score(x_torsion_train, y_torsion_train)
print('Accuracy of logistic regression classifier on train set:', score_torsion_train)
score_torsion_test = lr_torsion.score(x_torsion_test, y_torsion_test)
print('Accuracy of logistic regression classifier on test set:', score_torsion_test)
score_torsion_val = lr_torsion.score(x_torsion_val, y_torsion_val)
print('Accuracy of logistic regression classifier on validation set:', score_torsion_val)
print(classification_report(y_torsion_val, predictions_torsion_val))

[[144   8]
 [ 21 127]]
[[130   2]
 [ 20 103]]
Accuracy of logistic regression classifier on train set: 0.9882271468144044
Accuracy of logistic regression classifier on test set: 0.9033333333333333
Accuracy of logistic regression classifier on validation set: 0.9137254901960784
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       132
           1       0.98      0.84      0.90       123

    accuracy                           0.91       255
   macro avg       0.92      0.91      0.91       255
weighted avg       0.92      0.91      0.91       255



In [74]:
knn = KNeighborsClassifier()
grid_params = {'n_neighbors': [3, 5, 7], 'weights': ['distance'], 'metric': ['euclidean']}
knn_cv = GridSearchCV(knn, grid_params, cv=2, verbose = 1, n_jobs = -1)
knn_morg = knn_cv.fit(x_morg_train, y_morg_train)
knn_cv.best_params_
knn_cv.best_score_

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  3.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  3.0min finished


0.6877214006843636

In [75]:
knn_3n = KNeighborsClassifier(n_neighbors=3)
knn_AP = knn_3n.fit(x_AP_train, y_AP_train)
knn_5n = KNeighborsClassifier(n_neighbors=5)
knn_AP5 = knn_5n.fit(x_AP_train, y_AP_train)
knn_7n = KNeighborsClassifier(n_neighbors=7)
knn_AP7 = knn_7n.fit(x_AP_train, y_AP_train)
knn_AP_3n = knn_AP.score(x_AP_test, y_AP_test)
print(knn_AP_3n)
knn_AP_5n = knn_AP5.score(x_AP_test, y_AP_test)
print(knn_AP_5n)
knn_AP_7n = knn_AP7.score(x_AP_test, y_AP_test)
print(knn_AP_7n)


0.7966666666666666
0.7533333333333333
0.7433333333333333


In [77]:
knn3 = KNeighborsClassifier(n_neighbors=3)
knn_morg = knn3.fit(x_morg_train, y_morg_train)
knn_rd = knn3.fit(x_rd_train, y_rd_train)
knn_AP = knn3.fit(x_AP_train, y_AP_train)
knn_torsion = knn3.fit(x_torsion_train, y_torsion_train)
def mod_acc_knn(x_train, y_train, x_test, y_test):
    knn3 = KNeighborsClassifier(n_neighbors=3)
    knn_fp = knn3.fit(x_train, y_train)
    score_fp_test_knn = knn_fp.score(x_test, y_test)
    return score_fp_test_knn

In [78]:
knn_morg_acc_train = mod_acc_knn(x_morg_train, y_morg_train, x_morg_train, y_morg_train)
print(knn_morg_acc_train)
knn_rd_acc_train = mod_acc_knn(x_rd_train, y_rd_train, x_rd_train, y_rd_train)
print(knn_rd_acc_train)
knn_AP_acc_train = mod_acc_knn(x_AP_train, y_AP_train, x_AP_train, y_AP_train)
print(knn_AP_acc_train)
knn_torsion_acc_train = mod_acc_knn(x_torsion_train, y_torsion_train, x_torsion_train, y_torsion_train)
print(knn_torsion_acc_train)

0.8558682223747426
0.8711484593837535
0.8885041551246537
0.8552631578947368


In [79]:
knn_morg_acc_test = mod_acc_knn(x_morg_train, y_morg_train, x_morg_test, y_morg_test)
print(knn_morg_acc_test)
knn_rd_acc_test = mod_acc_knn(x_rd_train, y_rd_train, x_rd_test, y_rd_test)
print(knn_rd_acc_test)
knn_AP_acc_test = mod_acc_knn(x_AP_train, y_AP_train, x_AP_test, y_AP_test)
print(knn_AP_acc_test)
knn_torsion_acc_test = mod_acc_knn(x_torsion_train, y_torsion_train, x_torsion_test, y_torsion_test)
print(knn_torsion_acc_test)

0.7722772277227723
0.8215488215488216
0.7966666666666666
0.7666666666666667


In [80]:
knn_morg_acc_val = mod_acc_knn(x_morg_train, y_morg_train, x_morg_val, y_morg_val)
print(knn_morg_acc_val)
knn_rd_acc_val = mod_acc_knn(x_rd_train, y_rd_train, x_rd_val, y_rd_val)
print(knn_rd_acc_val)
knn_AP_acc_val = mod_acc_knn(x_AP_train, y_AP_train, x_AP_val, y_AP_val)
print(knn_AP_acc_val)
knn_torsion_acc_val = mod_acc_knn(x_torsion_train, y_torsion_train, x_torsion_val, y_torsion_val)
print(knn_torsion_acc_val)

0.751937984496124
0.7817460317460317
0.807843137254902
0.7490196078431373


In [86]:
def auc_calc(x_val, y_val, model):
    prob = model.predict_proba(x_val)
    prob = prob[:, 1]
    auc = metrics.roc_auc_score(y_val, prob)
    return 'AUC: %.6f' % auc

In [87]:
morg_lr_auc = auc_calc(x_morg_val, y_morg_val, lr_morg)
print("AUC_logistic regression: ", morg_lr_auc)
morg_knn_auc = auc_calc(x_morg_val, y_morg_val, knn_morg)
print("AUC_k-nearest naighbor: ", morg_knn_auc)


AUC_logistic regression:  AUC: 0.976282
AUC_k-nearest naighbor:  AUC: 0.355396


In [88]:
rd_lr_auc = auc_calc(x_rd_val, y_rd_val, lr_rd)
print("AUC_logistic regression: ", rd_lr_auc)
rd_knn_auc = auc_calc(x_rd_val, y_rd_val, knn_rd)
print("AUC_k-nearest neighbor: ", rd_knn_auc)


AUC_logistic regression:  AUC: 0.953216
AUC_k-nearest neighbor:  AUC: 0.487859


In [89]:
AP_lr_auc = auc_calc(x_AP_val, y_AP_val, lr_AP)
print("AUC_logistic regression: ", AP_lr_auc)
AP_knn_auc = auc_calc(x_AP_val, y_AP_val, knn_AP)
print("AUC_k-nearest naighbor: ", AP_knn_auc)


AUC_logistic regression:  AUC: 0.964400
AUC_k-nearest naighbor:  AUC: 0.481245


In [90]:
torsion_lr_auc = auc_calc(x_torsion_val, y_torsion_val, lr_torsion)
print("AUC_logistic regression: ", torsion_lr_auc)
torsion_knn_auc = auc_calc(x_torsion_val, y_torsion_val, knn_torsion)
print("AUC_k-nearest naighbor: ", torsion_knn_auc)


AUC_logistic regression:  AUC: 0.918576
AUC_k-nearest naighbor:  AUC: 0.805525


In [91]:
def f1score(x_val, y_val, model):
    predictions = model.predict(x_val)
    clrpt = classification_report(y_val, predictions, output_dict = True)
    d = clrpt["weighted avg"]
    f1 = d['f1-score']
    return 'f1-score %.6f' % f1
morg_lr_f1 = f1score(x_morg_val, y_morg_val, lr_morg)
morg_knn_f1 = f1score(x_morg_val, y_morg_val, knn_morg)
print(morg_lr_f1)
print(morg_knn_f1)


f1-score 0.926628
f1-score 0.313015


In [92]:
rd_lr_f1 = f1score(x_rd_val, y_rd_val, lr_rd)
rd_knn_f1 = f1score(x_rd_val, y_rd_val, knn_rd)
print(rd_lr_f1)
print(rd_knn_f1)


f1-score 0.908867
f1-score 0.323342


In [93]:
AP_lr_f1 = f1score(x_AP_val, y_AP_val, lr_AP)
AP_knn_f1 = f1score(x_AP_val, y_AP_val, knn_AP)
print(AP_lr_f1)
print(AP_knn_f1)


f1-score 0.933218
f1-score 0.374485


In [94]:
torsion_lr_f1 = f1score(x_torsion_val, y_torsion_val, lr_torsion)
torsion_knn_f1 = f1score(x_torsion_val, y_torsion_val, knn_torsion)
print(torsion_lr_f1)
print(torsion_knn_f1)


f1-score 0.913073
f1-score 0.735035
